In [3]:
import re
import csv
import json
import chess
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import islice
from datasets import load_dataset


import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset

# Data Arrangement

In [134]:
ds = load_dataset("austindavis/lichess-uci", split="train", streaming=True)

In [157]:
def extract_positions_and_evals(moves_str):
    move_san, fen_positions = [], []
    board = chess.Board()
    moves = moves_str.split()
    for pose in moves:
        try:
            fen_positions.append(board.fen())
            board.push_san(pose)
        except:
            return moves, fen_positions

    return moves, fen_positions

In [136]:
sample_iter = islice(ds, 10_000)
sample_list = list(sample_iter)

In [158]:
all_moves, positions, labels, game_ids = [], [], [], []
for game_id, row in tqdm(enumerate(sample_list)):
    moves_list, fen_list = extract_positions_and_evals(row["Transcript"])
    positions.extend(fen_list)
    all_moves.extend(moves_list[:len(fen_list)])
df = pd.DataFrame({"fen": positions, "move": all_moves})
# df.to_csv("chess_positions.csv", index=False)

10000it [00:19, 515.15it/s]

0.9838


# Fitting The Data To The Model

In [ ]:
df = pd.read_csv("chess_positions.csv")

In [120]:
piece_map = {'P':0,'N':1,'B':2,'R':3,'Q':4,'K':5,
             'p':6,'n':7,'b':8,'r':9,'q':10,'k':11}

def fen_to_tensor(fen):
    board = chess.Board(fen)
    mat = np.zeros((8,8,12), dtype=np.float32)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            row = 7 - chess.square_rank(square)
            col = chess.square_file(square)
            mat[row, col, piece_map[piece.symbol()]] = 1.0
    turn_channel = np.full((8,8,1), int(board.turn), dtype=np.float32)
    mat = np.concatenate([mat, turn_channel], axis=-1)
    return mat

In [121]:
all_moves = sorted(set(df["move"].to_list()))
move2idx = {m:i for i,m in enumerate(all_moves)}
idx2move = {i:m for m,i in move2idx.items()}

X_list = [fen_to_tensor(fen) for fen in df["fen"]]

X = torch.tensor(np.array(X_list), dtype=torch.float32).permute(0,3,1,2)
y_move = torch.tensor([move2idx[m] for m in df["move"]], dtype=torch.long)


In [ ]:
torch.save({"X": X, "y_move": y_move}, "data.pt")

with open('my_dict.json', 'w') as f:
    json.dump(move2idx, f)

# Training

In [68]:
import torch
import torch.nn as nn

class ChessMovePredictor(nn.Module):
    def __init__(self, num_moves):
        super().__init__()
        self.conv1 = nn.Conv2d(13, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 4 * 4, 256)
        self.fc_move = nn.Linear(256, num_moves)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        return self.fc_move(x)

In [13]:
data = torch.load("data.pt")
X = data["X"]
y_move = data["y_move"]

with open('move_idx.json', 'r') as f:
    move2idx = json.load(f)

In [64]:
dataset = TensorDataset(X, y_move)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ChessMovePredictor(num_moves=len(move2idx)).to(device)

criterion_move = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
for epoch in range(5):
    model.train()
    total_loss = 0.0
    total_samples = 0

    for X_batch, y_move_batch in loader:
        X_batch = X_batch.to(device)
        y_move_batch = y_move_batch.to(device)

        optimizer.zero_grad()
        logits_move = model(X_batch)

        loss = criterion_move(logits_move, y_move_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * X_batch.size(0)
        total_samples += X_batch.size(0)

    epoch_loss = total_loss / total_samples
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")


In [ ]:
torch.save(model.state_dict(), "chess_model.pth")

# extend move2idx

In [ ]:
df_old = pd.read_csv("chess_positions.csv")
df_new = pd.read_csv("chess_positions_new.csv")

In [ ]:
all_moves = sorted(set(df_old["move"]) | set(df_new["move"]))
move2idx = {m: i for i, m in enumerate(all_moves)}
idx2move = {i: m for m, i in move2idx.items()}

In [ ]:
y_move_old = torch.tensor([move2idx[m] for m in df_old["move"]])
y_move_new = torch.tensor([move2idx[m] for m in df_new["moves"]])

In [ ]:
y_eval_old = torch.tensor([0]*len(df_old), dtype=torch.float32).unsqueeze(1)
y_eval_new = torch.tensor(df_new["score"].values, dtype=torch.float32).unsqueeze(1)

In [ ]:
# data = torch.load("/content/drive/MyDrive/python/chess_dataset/data.pt")
# X_old = data["X"]
X_list = [fen_to_tensor(fen) for fen in df_old["fen"]]
X_old = torch.tensor(np.array(X_list), dtype=torch.float32).permute(0,3,1,2)
X_list = [fen_to_tensor(fen) for fen in df_new["fen"]]
X_new = torch.tensor(np.array(X_list), dtype=torch.float32).permute(0,3,1,2)

In [ ]:
X_total = torch.cat([X_old, X_new])
y_move_total = torch.cat([y_move_old, y_move_new])
y_eval_total = torch.cat([y_eval_old, y_eval_new])

In [ ]:
# two heads model
import torch
import torch.nn as nn

class ChessMovePredictor(nn.Module):
    def __init__(self, num_moves):
        super().__init__()
        self.conv1 = nn.Conv2d(13, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 4 * 4, 256)

        # שני ראשים – אחד לחיזוי מהלך, אחד להערכת מצב
        self.fc_move = nn.Linear(256, num_moves)
        self.fc_eval = nn.Linear(256, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))

        move_logits = self.fc_move(x)
        eval_pred = torch.tanh(self.fc_eval(x)) 
        return move_logits, eval_pred

In [ ]:
with open('/content/drive/MyDrive/python/chess_dataset/move_idx.json', 'r') as f:
    old_move2idx = json.load(f)

In [ ]:
new_num_moves = len(move2idx)
model = ChessMovePredictor(num_moves=new_num_moves)
old_state = torch.load("/content/drive/MyDrive/python/chess_dataset/chess_model_games.pth", map_location="cpu")

with torch.no_grad():
    for name, param in model.state_dict().items():
        if name in old_state and old_state[name].shape == param.shape and "fc_move" not in name:
            param.copy_(old_state[name])

old_num_moves = old_state["fc_move.weight"].shape[0]
copy_size = min(old_num_moves, new_num_moves)

with torch.no_grad():
    model.fc_move.weight[:copy_size] = old_state["fc_move.weight"][:copy_size]
    model.fc_move.bias[:copy_size] = old_state["fc_move.bias"][:copy_size]

In [ ]:
dataset = TensorDataset(X_total, y_move_total, y_eval_total)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion_move = nn.CrossEntropyLoss()
criterion_eval = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
lambda_eval = 0.3
for epoch in range(3):
    model.train()
    total_loss = 0.0
    total_samples = 0

    for X_batch, y_move_batch, y_eval_batch in loader:
        X_batch = X_batch.to(device)
        y_move_batch = y_move_batch.to(device)
        y_eval_batch = y_eval_batch.view(-1).to(device)

        optimizer.zero_grad()
        logits_move, pred_eval = model(X_batch)

        loss_move = criterion_move(logits_move, y_move_batch)
        loss_eval = criterion_eval(pred_eval.squeeze(), y_eval_batch)

        loss = loss_move + lambda_eval * loss_eval
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * X_batch.size(0)
        total_samples += X_batch.size(0)
    epoch_loss = total_loss / total_samples
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/python/chess_dataset/chess_model_games.pth")

In [87]:
def end_game(board):
    return board.is_game_over() or board.is_stalemate() or board.is_fifty_moves() or board.is_insufficient_material()

    


end_game(board)

False

In [152]:
board = chess.Board()

In [153]:
for i in range(51):
    legal_moves = list(board.legal_moves)
    move = legal_moves[random.randint(0, len(legal_moves)-1)]
    board.push(move)

In [134]:
board.is_fifty_moves()

True

In [135]:
board.is_game_over()

True

In [136]:
board.is_insufficient_material()

False

In [137]:
board.is_stalemate()

False

In [139]:
move2idx = {"a1a2": 0, "a1a3": 1, "a1a4": 2, "a1a5": 3, "a1a6": 4, "a1a7": 5, "a1a8": 6, "a1b1": 7, "a1b2": 8, "a1b3": 9, "a1c1": 10, "a1c2": 11, "a1c3": 12, "a1d1": 13, "a1d4": 14, "a1e1": 15, "a1e5": 16, "a1f1": 17, "a1f6": 18, "a1g1": 19, "a1g7": 20, "a1h1": 21, "a1h8": 22, "a2a1": 23, "a2a1N": 24, "a2a1Q": 25, "a2a1R": 26, "a2a1b": 27, "a2a1q": 28, "a2a1r": 29, "a2a3": 30, "a2a4": 31, "a2a5": 32, "a2a6": 33, "a2a7": 34, "a2a8": 35, "a2b1": 36, "a2b1Q": 37, "a2b2": 38, "a2b3": 39, "a2b4": 40, "a2c1": 41, "a2c2": 42, "a2c3": 43, "a2c4": 44, "a2d2": 45, "a2d5": 46, "a2e2": 47, "a2e6": 48, "a2f2": 49, "a2f7": 50, "a2g2": 51, "a2g8": 52, "a2h2": 53, "a3a1": 54, "a3a2": 55, "a3a4": 56, "a3a5": 57, "a3a6": 58, "a3a7": 59, "a3a8": 60, "a3b1": 61, "a3b2": 62, "a3b3": 63, "a3b4": 64, "a3b5": 65, "a3c1": 66, "a3c2": 67, "a3c3": 68, "a3c4": 69, "a3c5": 70, "a3d3": 71, "a3d6": 72, "a3e3": 73, "a3e7": 74, "a3f3": 75, "a3f8": 76, "a3g3": 77, "a3h3": 78, "a4a1": 79, "a4a2": 80, "a4a3": 81, "a4a5": 82, "a4a6": 83, "a4a7": 84, "a4a8": 85}

In [145]:
list(board.legal_moves)[0]

Move.from_uci('g1h3')

In [148]:
board.piece_at(list(board.legal_moves)[0].from_square).symbol()

'N'

In [154]:
board.checkers

<bound method Board.checkers of Board('4kbnr/rp2p3/2p1q2p/p1P1pbP1/Pn3p1P/1P3P2/3P4/R1BK1B1R b - - 2 26')>